
---

## 📈 **15. Monitoring & Logging**

---

### 📊 **Docker Metrics**

Docker exposes container metrics for CPU, memory, I/O, and network through:

* `docker stats`
* `docker inspect <container>` for detailed metadata
* `/var/lib/docker/containers/<id>/*-json.log` for raw logs (Linux)

---

### 📈 **Integrating with Prometheus + Grafana**

#### 🔧 Setup:

1. **Prometheus** scrapes Docker metrics using cAdvisor.
2. **Grafana** visualizes the data from Prometheus.

#### 🚀 Steps:

* Run cAdvisor:

  ```bash
  docker run \
    --volume=/:/rootfs:ro \
    --volume=/var/run:/var/run:rw \
    --volume=/sys:/sys:ro \
    --volume=/var/lib/docker/:/var/lib/docker:ro \
    --publish=8080:8080 \
    --detach=true \
    --name=cadvisor \
    gcr.io/cadvisor/cadvisor
  ```

* Configure Prometheus to scrape `localhost:8080`.

* Connect Grafana to Prometheus and import Docker dashboards.

---

### 📦 **ELK Stack (Elasticsearch + Logstash + Kibana)**

Used for log aggregation and search across containers.

1. **Filebeat** or **Fluentd** collects Docker logs.
2. Logs shipped to **Logstash**, parsed, and stored in **Elasticsearch**.
3. **Kibana** provides visualization.

---

### 📡 `docker events`

Real-time event stream of Docker daemon activities:

```bash
docker events
```

* Tracks actions like container start/stop, image pulls, etc.
* Useful for automation and debugging

---

